# To do

To fix:
* Fix imputing values, need to be really careful here. Clean up modes for fantasy positions for IR players
* Fix college stats pull

Can do with current data:
* Need to go through and 100% make sure imputing all the correct values in the last stage
* Make team change metric based on adp database which has latest team, compare to profootball focus team
* Need metric for number of net number of targets and rushing attempts leaving/joining the team
    * ex: A player was the number 2 guy on a team where the number 1 guy gets traded
    * ex: A player was the number 1 guy but a high volume guy comes in
* Need to integrate the rookie analysis in the targets joining vs not joining
    * Does the team they were drafted by have a star at the same position?
    * Where were they picked in the draft?
    * Winning pct of team picked?
    * Typical college metrics including injury history

Requires new scraping:
* Find better ADP source that goes back farther in years
    * This will fix lots of problems, limiting source of data too per year
    * Also missing data in some years that we have, it's a problem
* Add type of injury info, dummy variables for different injuries in the previous year(s)
* Get game level data: look at second half performance and how it affects ADP, a momentum metric
* New coach flag, maybe a coach name dummy variable or a quarter back name dummy

# Import packages and define functions

In [1]:
###################### Import Packages #############################################
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import statsmodels.formula.api as smf
from ffb_functions import *
pd.set_option('display.max_columns', 500)
pd.options.display.max_rows = 999


# Assemble pro football focus database and create metrics

In [47]:
############## Assemble the database #########################################
try:
    database = pd.read_csv('profootballfocus.csv')
except:
    database = data_assembly(2004, 2019)                            # can go back to at least 2000, limited by adp data
    database.to_csv('profootballfocus.csv', index = False)          # save original db

## change columns to floats for metric creation
cols_to_change = ['VBD', 'PosRank', 'OvRank', 'Rk'
                  , 'Age'
                  , 'PPR'
                  , 'GS', 'G'
                  , 'PaTD', 'Cmp', 'Int'
                  , 'RuTD', 'ReTD'
                  , 'PaYds', 'RuYds', 'ReYds', 'Rec'
                  , 'RuY/A', 'ReYds/R'
                  , 'PaAtt', 'RuAtt', 'Tgt'
                  , '2PM', '2PP', 'Fmb', 'FL', 'TD.3']
for col in cols_to_change:
    database[col] = database[col].astype(float)
    
## create new metrics #############################################
# points next year for regressions
shifted = shift_col(database, 'pts_next_year', 'PPR', -1)
shifted['pts_next_year'] = shifted['pts_next_year'].replace(np.nan, 0)                # impute 0 if they arent in top 500 next year

## impute stuff early #####
shifted['VBD'] = shifted['VBD'].replace(np.nan, 999)
shifted['OvRank'] = shifted['OvRank'].replace(np.nan, 999)

# games next year for regressions
shifted = shift_col(shifted, 'g_next_year', 'G', -1)                                  # create G_next_year col for regressions
shifted['g_next_year'] = shifted['g_next_year'].replace(np.nan, 0)                    # impute 0 if they arent in top 500 next year

# games previous years for injury history
shifted = shift_col(shifted, 'G_prev_year', 'G', 1) 
shifted = shift_col(shifted, 'G_prev_year_prev_year', 'G_prev_year', 1)               

# create ppg next year for y var
shifted['ppg_next_year'] = shifted['pts_next_year'] / shifted['g_next_year']
shifted['ppg_next_year'] = shifted['ppg_next_year'].replace(np.nan, 0)
shifted['ppg_this_year'] = shifted['PPR'] / shifted['G']                              # current year ppg
shifted['delta_ppg'] = shifted['ppg_next_year'] - shifted['ppg_this_year']            # change in ppg

# create injury history metrics
shifted['season_frac_1'] = shifted['G'] / 16                                          # 1 year injury history
shifted['season_frac_2'] = (shifted['G'] + shifted['G_prev_year']) / 32               # 2 year injury history
shifted['season_frac_3'] = (shifted['G'] + shifted['G_prev_year_prev_year']) / 48     # 3 year injury history

# create per attempt metrics to reduce multicollinearity
shifted['ReYds_per_R'] = shifted['ReYds/R']                                           # ReYds per reception
shifted['RuYds_per_A'] = shifted['RuY/A']                                             # RuYds per attempt
shifted['RuTD_per_Att'] = shifted['RuTD'] / shifted['RuAtt']                          # RuTds per attempt
shifted['PaYds_per_PaAtt'] = shifted['PaYds'] / shifted['PaAtt']                      # PaYds per attempt
shifted['PaTD_per_PaAtt'] = shifted['PaTD'] / shifted['PaAtt']                        # PaTds per attempt
shifted['Cmp_per_PaAtt'] = shifted['Cmp'] / shifted['PaAtt']                          # Completions per attempt
shifted['Int_per_PaAtt'] = shifted['Int'] / shifted['PaAtt']                          # Interceptions per attempt
shifted['Rec_per_tgt'] = shifted['Rec'] / shifted['Tgt']                              # Receptions per target
shifted['ReTD_per_rec'] = shifted['ReTD'] / shifted['Rec']                            # ReTD's per reception

# fraction of games played that they started
shifted['start_frac'] = shifted['GS'] / shifted['G']

# Create some per game metrics
for metric in ['Tgt', 'PaAtt', 'RuAtt']:
    new_str = metric + '_per_game'
    shifted[new_str] = shifted[metric] / shifted['G']
    
## share of team's fantasy points
# get fantasy points of team
gb = shifted.groupby(['Tm', 'Year']).sum().reset_index()[['Tm', 'Year', 'PPR']]
gb = gb[(gb.Tm != '2TM') & (gb.Tm != '3TM') & (gb.Tm != '4TM')]
# compare fantasy points of player to fantasy points of team
shifted['Fant_Share'] = 0
for i in range(len(shifted)):
    if 'TM' not in shifted.Tm[i]:
        temp_tm = shifted.loc[i, 'Tm']
        temp_yr = shifted.loc[i, 'Year']
        gb_val = gb.loc[(gb.Tm == temp_tm) & (gb.Year == temp_yr), 'PPR']
        shifted.loc[i, 'Fant_Share'] = shifted.loc[i, 'PPR'] / gb_val.iloc[0]
    else:
        shifted.loc[i, 'Fant_Share'] = 0 #this isnt technically true, may need to change this
        
# impute season frac by position, impute games prev by position
cols = ['season_frac_2', 'season_frac_3', 'G_prev_year', 'G_prev_year_prev_year']
for i in cols:
    shifted[i] = shifted[i].astype(float)
    shifted[i] = shifted[i].fillna(shifted.groupby('FantPos')[i].transform('mean'))

# some players for IR reasons dont have a fantasy position for one year
# need to fill in with their other fantasy positions from other years
shifted.loc[shifted.Name == 'Travis Kelce', 'FantPos'] = 'TE' #on the IR his first year, positionless in the stats
shifted.loc[shifted.Name == 'Sam Bradford', 'FantPos'] = 'QB' #on the IR one year
shifted.loc[shifted.Name == 'Chad Johnson', 'FantPos'] = 'WR' #on the IR one year
shifted.loc[shifted.Name == 'Marvin Jones', 'FantPos'] = 'WR' #on the IR one year
shifted.loc[shifted.Name == 'Brandon Coleman', 'FantPos'] = 'WR' #on the IR one year
shifted.loc[shifted.Name == 'Chris Givens', 'FantPos'] = 'WR' #on the IR one year
shifted.loc[shifted.Name == 'Vincent Brown', 'FantPos'] = 'WR' #on the IR one year

#shifted['FantPos'] = shifted['FantPos'].fillna(shifted.groupby('FantPos')[i].transform('mean'))
# hopefully figure out a clever way to impute modes here. for now, fix this thing

# impute all other stats with 0, because remaining nans are from 0 passes etc
shifted = shifted.fillna(0)
shifted = shifted.loc[shifted.Rk < 301].reset_index(drop = True) # keep relevant players

# save output so far
shifted.to_csv('profootballfocus_withmetrics.csv', index = False)

In [48]:
shifted.groupby('Year').count()
#database

,Rk,Name,Tm,FantPos,Age,G,GS,Cmp,PaAtt,PaYds,PaTD,Int,RuAtt,RuYds,RuY/A,RuTD,Tgt,Rec,ReYds,ReYds/R,ReTD,Fmb,FL,TD.3,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,pts_next_year,g_next_year,G_prev_year,G_prev_year_prev_year,ppg_next_year,ppg_this_year,delta_ppg,season_frac_1,season_frac_2,season_frac_3,ReYds_per_R,RuYds_per_A,RuTD_per_Att,PaYds_per_PaAtt,PaTD_per_PaAtt,Cmp_per_PaAtt,Int_per_PaAtt,Rec_per_tgt,ReTD_per_rec,start_frac,Tgt_per_game,PaAtt_per_game,RuAtt_per_game,Fant_Share
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300
2005,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300
2006,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300
2007,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300
2008,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300
2009,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300
2010,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300
2011,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300
2012,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300


# Assemble and merge ADP database (ppr)

In [50]:
## new assembly gets us many more observations per year and more years, in return lose std dev variable
try:
    adp_frame_0 = pd.read_csv('adp_2.csv')
except:
    adp_frame_0 = new_assembly(2004, 2020)         # sometimes this link is a problem. to fix it, open up the link in a browser and run again
    adp_frame_0.to_csv('adp_2.csv', index = False)
adp_frame = adp_frame_0
adp_frame['Year'] = adp_frame['Year'] - 1
adp_frame['Tm'] = adp_frame['Tm'].str.replace('JAC', 'JAX')
adp_frame['Tm'] = adp_frame['Tm'].str.replace('KCC', 'KAN')
adp_frame['Tm'] = adp_frame['Tm'].str.replace('GBP', 'GNB')
adp_frame['Tm'] = adp_frame['Tm'].str.replace('NOS', 'NOR')


# Dont include pos, too many differences of wr's at te's
# merge the databases, inner bc only want fantasy relevant players
frame_w_adp = shifted.merge(adp_frame, on = ['Name', 'Year'], how = 'inner')

frame_w_adp = frame_w_adp.rename(columns = {'Tm_x': 'Old_Team', 'Tm_y':'New_Team'})

# flag if team in adp database is different from pff database, means switched teams
frame_w_adp['Tm_change_flag'] = frame_w_adp.apply(\
                                             lambda x: new_team(x['Old_Team'], x['New_Team']), axis = 1)

#frame_w_adp['New_Team'] = frame_w_adp['New_Team'].replace(np.nan, 'Undrafted')

# dont include unranked because that includes rookies. Separate analysis for them
frame_w_adp = frame_w_adp.loc[pd.notna(frame_w_adp.Rk)].reset_index(drop = True)
# hard code in Mike Williams problem eventually
frame_w_adp = frame_w_adp.loc[frame_w_adp.Name != 'Mike Williams'].reset_index(drop = True)

########### keep this bit because STD DEV is an important variable ###################
try:
    adp_frame_alt = pd.read_csv('adp_8.csv')
except:
    adp_frame_alt = ADP_assembly(2010, 2020)         # sometimes this link is a problem. to fix it, open up the link in a browser and run again
    adp_frame_alt.to_csv('adp_.csv', index = False)
    
adp_frame_alt_2 = adp_frame_alt[['Name', 'Year', 'Std.Dev']]

# the year is off of the profootball focus data by 1
adp_frame_alt_2['Year'] = adp_frame_alt_2['Year'] - 1

adp_frame_alt_2 = adp_frame_alt_2.replace('Odell Beckham Jr', 'Odell Beckham')

# maybe merge on pos too? cant merge on team because players switch from profootballfocus to adp
# merge the databases, inner bc only want fantasy relevant players
frame_w_adp_2 = frame_w_adp.merge(adp_frame_alt_2, on = ['Name', 'Year'], how = 'left')

frame_w_adp_2['Std.Dev'] = frame_w_adp_2['Std.Dev'].fillna(frame_w_adp_2['Std.Dev'].mean())
frame_w_adp = frame_w_adp_2

/Users/JoshCoven/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [52]:
frame_w_adp.groupby('Year').count()

,Rk,Name,Old_Team,FantPos,Age,G,GS,Cmp,PaAtt,PaYds,PaTD,Int,RuAtt,RuYds,RuY/A,RuTD,Tgt,Rec,ReYds,ReYds/R,ReTD,Fmb,FL,TD.3,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,pts_next_year,g_next_year,G_prev_year,G_prev_year_prev_year,ppg_next_year,ppg_this_year,delta_ppg,season_frac_1,season_frac_2,season_frac_3,ReYds_per_R,RuYds_per_A,RuTD_per_Att,PaYds_per_PaAtt,PaTD_per_PaAtt,Cmp_per_PaAtt,Int_per_PaAtt,Rec_per_tgt,ReTD_per_rec,start_frac,Tgt_per_game,PaAtt_per_game,RuAtt_per_game,Fant_Share,New_Team,Overall,Tm_change_flag,Std.Dev
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192,192
2005,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200
2006,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201
2007,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202
2008,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194,194
2009,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206,206
2010,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213,213
2011,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182
2012,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182


# Assemble and merge college database and draft/combine database

In [53]:
# college data pull 1: college stats
## college statistics db, school, conf, games, stats, year ######
try:
    college_0 = pd.read_csv('college_stats.csv')
except:
    college_0 = college_assembly(2000, 2020)
    college_0.to_csv('college_stats.csv', index = False)
    
college = college_0.fillna(0) # this fills in stats that are missing, works
college['Name'] = college['Name'].str.replace('Joshua Jacobs', 'Josh Jacobs') # make names match across databases


# college data pull 2: draft pick
## draft pick database, college, age, some stats
try:
    rk_0 = pd.read_csv('draft_pick.csv')
except:
    rk_0 = rookie_assembly(2000, 2020)
    rk_0.to_csv('draft_pick.csv', index = False)
    
rk = rk_0[['Pick', 'Tm', 'Name', 'Age', 'College', 'Year', 'FantPos']]
# keep fantasy relevant positions
rk = rk.loc[(rk.FantPos == 'QB') 
         | (rk.FantPos == 'WR')
        | (rk.FantPos == 'RB')
        | (rk.FantPos == 'TE')].reset_index(drop = True)


# college data pull 3: combine numbers
## school, height, weight, combine stats, position, name
try:
    comb_0 = pd.read_csv('combine.csv')
except:
    comb_0 = combine_assembly(2000, 2020)
    comb_0.to_csv('combine.csv', index = False)
    
comb = comb_0.rename(columns = {'Pos':'FantPos'}) # for merge
comb = comb[['Name', 'Year', 'School', 'FantPos', 'Ht', 'Wt', 'Dash', 'Vertical', 'Bench', 'Broad_Jump', 'Three_Cone', 'Shuttle']]
#keep fantasy relevant positions
comb = comb.loc[(comb.FantPos == 'QB') 
         | (comb.FantPos == 'WR')
        | (comb.FantPos == 'RB')
        | (comb.FantPos == 'TE')].reset_index(drop = True) 

comb['height'] = comb['Ht'].apply(lambda x: 12 * float(x.split('-')[0]) + float(x.split('-')[1]) \
                                          if isinstance(x, str) else np.nan)

# impute combine statistics based on position mean
cols = ['height', 'Wt', 'Dash', 'Vertical', 'Bench', 'Broad_Jump', 'Three_Cone', 'Shuttle']
for i in cols:
    comb[i] = comb[i].astype(float)
    comb[i] = comb[i].fillna(comb.groupby('FantPos')[i].transform('mean'))
    
# merge rookie names, draft capital with combine statistics
draft_and_combine = rk.merge(comb, on = ['Name', 'FantPos', 'Year'], how = 'outer')
draft_and_combine = draft_and_combine.loc[draft_and_combine.Name != 'Mike Williams'].reset_index(drop = True)

## merge rookie names and draft capital with college statistics
draft_combine_stats = draft_and_combine.merge(college.drop(columns = 'Year'), on = ['Name', 'School'], how = 'outer')
draft_combine_stats = draft_combine_stats.loc[draft_combine_stats.Name != 'Mike Williams'].reset_index(drop = True)

## merge all three college databases back to fantasy stats from profootballfocus
all_rookie = draft_combine_stats.merge(frame_w_adp[['Name', 'Year', 'Std.Dev', 'Overall', 'FantPos']], on = ['Name', 'Year', 'FantPos'], how = 'inner')
all_rookie = all_rookie.drop_duplicates(['Name', 'FantPos', 'Year'], keep = 'first').reset_index(drop = True)
## the only duplicate is Mike Williams. Need to drop him and hard code him in later. There were many fantasy relevant mike williams'
all_rookie = all_rookie.loc[all_rookie.Name != 'Mike Williams'].reset_index(drop = True)

all_rookie.to_csv('full_college_df.csv', index = False) # save so far


## for non-rookies, I really want to merge combine and draft pick data back to original frame_w_adp
## for rookies, I want to merge all this stuff but include college statistics

In [29]:
draft_and_combine

,Pick,Tm,Name,Age,College,Year,FantPos,School,Ht,Wt,Dash,Vertical,Bench,Broad_Jump,Three_Cone,Shuttle,height
0,4.0,CIN,Peter Warrick,23.0,Florida St.,2000,WR,Florida St.,5-11,194.0,4.580000,35.287858,14.556322,120.427972,6.972027,4.219032,71.0
1,5.0,BAL,Jamal Lewis,21.0,Tennessee,2000,RB,Tennessee,6-0,240.0,4.580000,34.404968,23.000000,118.065789,7.063914,4.265000,72.0
2,7.0,ARI,Thomas Jones,22.0,Virginia,2000,RB,Virginia,5-10,216.0,4.450000,34.404968,19.395455,118.065789,7.063914,4.265000,70.0
3,8.0,PIT,Plaxico Burress,23.0,Michigan St.,2000,WR,Michigan St.,6-5,231.0,4.590000,33.000000,14.556322,115.000000,6.972027,4.219032,77.0
4,10.0,BAL,Travis Taylor,22.0,Florida,2000,WR,Florida,6-1,199.0,4.430000,37.000000,14.556322,118.000000,7.150000,4.219032,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2487,NaN,NaN,Cody Thompson,NaN,NaN,2019,WR,Toledo,6-1,205.0,4.570000,38.500000,19.000000,119.000000,6.870000,4.030000,73.0
2488,NaN,NaN,Jamarius Way,NaN,NaN,2019,WR,South Alabama,6-3,215.0,4.630000,32.500000,16.000000,119.000000,6.972027,4.660000,75.0
2489,NaN,NaN,Alex Wesley,NaN,NaN,2019,WR,Northern Colorado,6-0,190.0,4.450000,38.500000,13.000000,125.000000,7.400000,4.440000,72.0
2490,NaN,NaN,Antoine Wesley,NaN,NaN,2019,WR,Texas Tech,6-4,206.0,4.511911,37.000000,6.000000,117.000000,7.070000,4.260000,76.0


In [59]:
for_redraft = draft_and_combine[['Name', 'Pick', 'FantPos', 'College', 'height', 'Wt', 'Dash', 'Vertical', 'Bench', 'Broad_Jump', 'Three_Cone'
      , 'Shuttle']]

final_frame = frame_w_adp.merge(for_redraft, on = ['Name', 'FantPos'], how = 'outer')
final_frame = final_frame.loc[pd.notna(final_frame.Old_Team)].reset_index(drop = True)
## positionally impute mean values for combine
cols = ['Wt', 'Dash', 'Vertical', 'Bench', 'Broad_Jump', 'Three_Cone', 'Shuttle']
for i in cols:
    final_frame[i] = final_frame[i].astype(float)
    final_frame[i] = final_frame[i].fillna(final_frame.groupby("FantPos")[i].transform('mean'))
## impute 999 for ranking variables
#cols = ['season_frac_2', 'season_frac_3', 'G_prev_year', 'G_prev_year_prev_year']
#for i in cols:
 #   shifted[i] = shifted[i].astype(float)
  #  shifted[i] = shifted[i].fillna(shifted.groupby('FantPos')[i].transform('mean'))


## impute 0's for everything else
final_frame = final_frame.fillna(0)
final_frame = final_frame.rename(columns = {'Year_x': 'Year'})
final_frame = final_frame.rename(columns = {'Tm': 'Other_Tm'})
final_frame = final_frame.rename(columns = {'Tm_x': 'Tm'})
final_frame = final_frame.rename(columns = {'Tm_y': 'New_Tm'})

final_frame
final_frame.to_csv('big_redraft_frame.csv', index = False)

In [60]:
final_frame.loc[final_frame.Shuttle == 0]
#for_redraft.loc[for_redraft.Name == 'Amari Cooper']
#frame_w_adp.loc[frame_w_adp.Name == 'Amari Cooper']
#adp_frame.loc[adp_frame.Name == 'Antonio Brown']
#test2.loc[test2.Name == 'Antonio Brown']

,Rk,Name,Old_Team,FantPos,Age,G,GS,Cmp,PaAtt,PaYds,PaTD,Int,RuAtt,RuYds,RuY/A,RuTD,Tgt,Rec,ReYds,ReYds/R,ReTD,Fmb,FL,TD.3,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,Year,pts_next_year,g_next_year,G_prev_year,G_prev_year_prev_year,ppg_next_year,ppg_this_year,delta_ppg,season_frac_1,season_frac_2,season_frac_3,ReYds_per_R,RuYds_per_A,RuTD_per_Att,PaYds_per_PaAtt,PaTD_per_PaAtt,Cmp_per_PaAtt,Int_per_PaAtt,Rec_per_tgt,ReTD_per_rec,start_frac,Tgt_per_game,PaAtt_per_game,RuAtt_per_game,Fant_Share,New_Team,Overall,Tm_change_flag,Std.Dev,Pick,College,height,Wt,Dash,Vertical,Bench,Broad_Jump,Three_Cone,Shuttle
1698,171.0,Kolby Smith,KAN,0,23.0,16.0,6.0,0.0,0.0,0.0,0.0,0.0,112.0,407.0,3.63,2.0,30.0,22.0,148.0,6.73,0.0,0.0,0.0,2.0,0.0,0.0,68.0,89.5,95.5,78.5,999.0,51.0,999.0,2007.0,31.2,7.0,0.0,0.0,4.457143,5.593750,-1.136607,1.0000,0.0,0.0,6.73,3.63,0.017857,0.0,0.0,0.0,0.0,0.733333,0.000000,0.375,1.875,0.0,7.000000,0.082359,KAN,201.07,0.0,9.684929,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2083,22.0,Mike Anderson,DEN,0,32.0,15.0,15.0,0.0,0.0,0.0,0.0,0.0,239.0,1014.0,4.24,12.0,21.0,18.0,212.0,11.78,1.0,2.0,0.0,13.0,0.0,0.0,201.0,218.6,224.6,209.6,58.0,10.0,22.0,2005.0,38.7,16.0,0.0,0.0,2.418750,14.573333,-12.154583,0.9375,0.0,0.0,11.78,4.24,0.050209,0.0,0.0,0.0,0.0,0.857143,0.055556,1.000,1.400,0.0,15.933333,0.164969,BAL,128.61,1.0,9.684929,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Assemble and merge data from teams to get winning pct

In [61]:
team_temp = team_assembly(2008, 2019)

tm_dict = {'Pittsburgh': 'PIT',
 'Philadelphia': 'PHI',
 'New England': 'NWE',
 'Minnesota': 'MIN',
 'Carolina': 'CAR',
 'LA Rams': 'LAR',
 'New Orleans': 'NOR',
 'Jacksonville': 'JAX',
 'Kansas City': 'KAN',
 'Atlanta': 'ATL',
 'LA Chargers': 'LAC',
 'Seattle': 'SEA',
 'Buffalo': 'BUG',
 'Dallas': 'DAL',
 'Tennessee': 'TEN',
 'Detroit': 'DET',
 'Baltimore': 'BAL',
 'Arizona': 'ARI',
 'Washington': 'WAS',
 'Green Bay': 'GNB',
 'Cincinnati': 'CIN',
 'Oakland': 'OAK',
 'San Francisco': 'SFO',
 'Miami': 'MIA',
 'Denver': 'DEN',
 'NY Jets': 'NYJ',
 'Tampa Bay': 'TAM',
 'Chicago': 'CHI',
 'Indianapolis': 'IND',
 'Houston': 'HOU',
 'NY Giants': 'NYG',
 'Cleveland': 'CLE'}

team_temp['Tm'] = team_temp['Team'].apply(lambda x: tm_dict[x])
team_temp = team_temp[['Tm', 'Win_PCT', 'Year']]
team_temp['Win_PCT'] = team_temp['Win_PCT'].apply(lambda x: float(x[0:len(x)-1]))

In [62]:
final_frame = final_frame.rename(columns = {'New_Team':'Tm'})
real_final = final_frame.merge(team_temp, on = ['Year', 'Tm'], how = 'outer')
real_final = real_final.loc[pd.notna(real_final.Rk)].reset_index(drop = True)
real_final.Win_PCT = real_final.Win_PCT.fillna(0.500)
real_final = real_final.loc[(real_final.FantPos == 'QB') 
         | (real_final.FantPos == 'WR')
        | (real_final.FantPos == 'RB')
        | (real_final.FantPos == 'TE')].reset_index(drop = True)
real_final.to_csv('final_frame_teams.csv', index = False)



In [ ]:
database = data_assembly(2000, 2005)                            # pull based on years

In [63]:
real_final

,Rk,Name,Old_Team,FantPos,Age,G,GS,Cmp,PaAtt,PaYds,PaTD,Int,RuAtt,RuYds,RuY/A,RuTD,Tgt,Rec,ReYds,ReYds/R,ReTD,Fmb,FL,TD.3,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,Year,pts_next_year,g_next_year,G_prev_year,G_prev_year_prev_year,ppg_next_year,ppg_this_year,delta_ppg,season_frac_1,season_frac_2,season_frac_3,ReYds_per_R,RuYds_per_A,RuTD_per_Att,PaYds_per_PaAtt,PaTD_per_PaAtt,Cmp_per_PaAtt,Int_per_PaAtt,Rec_per_tgt,ReTD_per_rec,start_frac,Tgt_per_game,PaAtt_per_game,RuAtt_per_game,Fant_Share,Tm,Overall,Tm_change_flag,Std.Dev,Pick,College,height,Wt,Dash,Vertical,Bench,Broad_Jump,Three_Cone,Shuttle,Win_PCT
0,35.0,A.J. Green,CIN,WR,23.0,15.0,15.0,0.0,0.0,0.0,0.0,0.0,5.0,53.0,10.60,0.0,115.0,65.0,1057.0,16.26,7.0,1.0,0.0,7.0,0.0,0.0,153.0,218.0,221.0,185.5,37.0,14.0,35.0,2011.0,299.8,16.0,12.830099,13.146263,18.737500,14.533333,4.204167,0.9375,0.799130,0.540244,16.26,10.60,0.000000,0.000000,0.00000,0.000000,0.000000,0.565217,0.107692,1.000000,7.666667,0.0000,0.333333,0.177988,CIN,27.23,0.0,4.600000,4.0,Georgia,76.0,211.000000,4.480000,34.500000,18.000000,126.000000,6.910000,4.210000,56.2
1,83.0,Andy Dalton,CIN,QB,24.0,16.0,16.0,300.0,516.0,3398.0,20.0,13.0,37.0,152.0,4.11,1.0,1.0,0.0,0.0,0.00,0.0,5.0,0.0,1.0,0.0,0.0,213.0,211.1,230.1,224.1,999.0,18.0,999.0,2011.0,250.8,16.0,9.140766,9.828737,15.675000,13.193750,2.481250,1.0000,0.564295,0.394231,0.00,4.11,0.027027,6.585271,0.03876,0.581395,0.025194,0.000000,0.000000,1.000000,0.062500,32.2500,2.312500,0.172355,CIN,134.25,0.0,9.684929,35.0,TCU,74.0,215.000000,4.830000,29.500000,19.210526,106.000000,6.930000,4.270000,56.2
2,91.0,BenJarvus Green-Ellis,NWE,RB,26.0,16.0,6.0,0.0,0.0,0.0,0.0,0.0,181.0,667.0,3.69,11.0,13.0,9.0,159.0,17.67,0.0,0.0,0.0,11.0,0.0,0.0,149.0,157.6,163.6,153.1,999.0,24.0,999.0,2011.0,173.8,15.0,16.000000,12.000000,11.586667,9.850000,1.736667,1.0000,1.000000,0.583333,17.67,3.69,0.060773,0.000000,0.00000,0.000000,0.000000,0.692308,0.000000,0.375000,0.812500,0.0000,11.312500,0.087633,CIN,67.72,1.0,5.700000,0.0,0,71.0,219.000000,4.600000,30.500000,24.000000,114.000000,7.063914,4.265000,56.2
3,193.0,Bernard Scott,CIN,RB,27.0,16.0,1.0,0.0,1.0,0.0,0.0,0.0,112.0,380.0,3.39,3.0,17.0,13.0,38.0,2.92,0.0,0.0,0.0,3.0,0.0,0.0,60.0,72.8,75.8,66.3,999.0,57.0,999.0,2011.0,3.5,2.0,16.000000,13.000000,1.750000,4.550000,-2.800000,1.0000,1.000000,0.604167,2.92,3.39,0.026786,0.000000,0.00000,0.000000,0.000000,0.764706,0.000000,0.062500,1.062500,0.0625,7.000000,0.059438,CIN,194.83,0.0,9.684929,209.0,Abilene Christian,70.0,200.000000,4.440000,36.000000,21.000000,125.000000,6.820000,4.080000,56.2
4,133.0,Jermaine Gresham,CIN,TE,23.0,14.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,92.0,56.0,596.0,10.64,6.0,1.0,0.0,6.0,0.0,0.0,96.0,151.6,154.6,123.6,999.0,13.0,999.0,2011.0,165.7,16.0,15.000000,13.127427,10.356250,10.828571,-0.472321,0.8750,0.906250,0.551831,10.64,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.608696,0.107143,0.928571,6.571429,0.0000,0.000000,0.123775,CIN,113.38,0.0,13.600000,21.0,Oklahoma,77.0,261.000000,4.660000,35.000000,20.000000,113.000000,7.070000,4.530000,56.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2930,139.0,Reggie Wayne,IND,WR,36.0,15.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,116.0,64.0,779.0,12.17,2.0,1.0,1.0,2.0,0.0,0.0,90.0,151.9,155.9,119.9,999.0,58.0,999.0,2014.0,0.0,0.0,7.000000,16.000000,0.000000,10.126667,-10.126667,0.9375,0.687500,0.645833,12.17,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.551724,0.031250,1.000000,7.733333,0.0000,0.000000,0.090164,FA,180.67,1.0,9.684929,30.0,Miami (FL),72.0,198.000000,4.450000,35.287858,14.556322,120.427972,6.972027,4.219032,0.5
2931,134.0,Trent Richardson,IND,RB,24.0,15.0,12.0,0.0,0.0,0.0,0.0,0.0,159.0,519.0,3.26,3.0,34.0,27.0,229.0,8.48,0.0,2.0,1.0,3.0,0.